# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,301.14,83,75,4.63,CC,1706982133
1,1,remire-montjoly,4.9167,-52.2667,305.17,62,20,5.66,GF,1706982117
2,2,tindouf,27.6711,-8.1474,294.40,8,0,7.20,DZ,1706982201
3,3,puerto ayora,-0.7393,-90.3518,304.13,79,55,5.02,EC,1706982201
4,4,grytviken,-54.2811,-36.5092,281.88,93,100,5.23,GS,1706982132


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x = 'Lng',
    y = 'Lat',
    c = 'Humidity',
    geo = True,
    title = 'City Map',
    xlabel = 'Longitude',
    ylabel = 'Latitude',
    frame_width = 700,
    frame_height = 500,
    cmap = 'viridis',
    colorbar = True,
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] > 20)

filtered_cities_df = city_data_df[criteria]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,tindouf,27.6711,-8.1474,294.40,8,0,7.20,DZ,1706982201
6,6,longyearbyen,78.2186,15.6401,264.06,67,0,6.69,SJ,1706982201
14,14,port lincoln,-34.7333,135.8667,292.42,67,0,1.81,AU,1706982123
21,21,kailua-kona,19.6406,-155.9956,294.13,90,0,14.92,US,1706982024
24,24,petropavlovsk-kamchatsky,53.0452,158.6483,253.48,85,0,4.00,RU,1706982203


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,83,
1,remire-montjoly,GF,4.9167,-52.2667,62,
2,tindouf,DZ,27.6711,-8.1474,8,
3,puerto ayora,EC,-0.7393,-90.3518,79,
4,grytviken,GS,-54.2811,-36.5092,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 5000  # You can adjust the radius as needed
params = {
    "radius": radius,
    "sort": "distance",  # Sorting results by distance
    "type": "hotel",  # Filter for hotels
    "limit": 1,  # Limiting to one result
}

# Set API key in the headers
headers = {
    "apikey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point({lon} {lat})"
    params["bias"] = f"point({lon} {lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary and headers
    response = requests.get(base_url, params=params, headers=headers)
    

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
west island - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
la ronge - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
ponta pora - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
alakurtti - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
fukuecho - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found


umm al quwain city - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
kourou - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
vorkuta - nearest hotel: No hotel found
yorosso - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
wynyard - nearest hotel: No hotel found
lorgues - nearest hotel: No hotel found
mae ramat - nearest hotel: No hotel found
lesogorsk - nearest hotel: No hotel found
bamboo flat - nearest hotel: No hotel found
sao paulo de olivenca - nearest hotel: No hotel found
manbij - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
raymondville - nearest hotel: No hotel found
businga - nearest hotel: No hotel found
san francisco de la paz - nearest hotel: No hotel found
college - nearest hotel: No hotel found
kattivakk

misliyah - nearest hotel: No hotel found
'ain benian - nearest hotel: No hotel found
vassouras - nearest hotel: No hotel found
mugia - nearest hotel: No hotel found
vetralla - nearest hotel: No hotel found
colchani - nearest hotel: No hotel found
natal - nearest hotel: No hotel found
buurhakaba - nearest hotel: No hotel found
dhuusamarreeb - nearest hotel: No hotel found
muisne - nearest hotel: No hotel found
lindisfarne - nearest hotel: No hotel found
leticia - nearest hotel: No hotel found
verkhovazh'ye - nearest hotel: No hotel found
savanna-la-mar - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
selfoss - nearest hotel: No hotel found
eyl - nearest hotel: No hotel found
lewistown - nearest hotel: No hotel found
mil'kovo - nearest hotel: No hotel found
tarija - nearest hotel: No hotel found
floro - nearest hotel: No hotel found
suntar - nearest hotel: No hotel found
mokhsogollokh - nearest hotel: No hotel found
bereznik - nearest hotel: No hotel found
taksimo - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,83,No hotel found
1,remire-montjoly,GF,4.9167,-52.2667,62,No hotel found
2,tindouf,DZ,27.6711,-8.1474,8,No hotel found
3,puerto ayora,EC,-0.7393,-90.3518,79,No hotel found
4,grytviken,GS,-54.2811,-36.5092,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)